In [21]:
import time
import requests
import json
import os
import unidecode
from unidecode import UnidecodeError
import tqdm
import urllib.parse
import shutil
import pickle

appids = [730, #CS:GO
          578080, #PUBG
          570, #Dota 2
          271590, #GTA V
          359550, #Rainbow Six Siege
          440, #Team Fortress 2
          105600, #Terraria
          4000, #Garry's Mod
          252490, #Rust
          292030, #The Witcher 3
          1172470, #Apex Legends
          945360, # Among Us
          1085660, #Destiny 2
          550, #Left 4 Dead 2
          1091500, #Cyberpunk 2077
          230410, #Warframe
          304930, #Unturned
          1245620, #Elden Ring
          346110, #ARK: Survival Evolved
          1097150, #Fall Guys
          381210, #Dead by Daylight
          413150, #Stardew Valley
          227300, #Euro Truck Simulator 2
          739630, #Phasmophobia
          252950, #Rocket League
          242760, #The Forest
          218620, #PAYDAY 2
          236390, #War Thunder
          1174180, #Red Dead Redemption 2
          892970, #Valheim
          444090, #Paladins
          291550, #Brawlhalla
          221100, #DayZ
          620, #Portal 2
          239140, #Dying Light
          322330, #Don't Starve Together
          367520, #Hollow Knight
          1172620, #Sea of Thieves
          582010, #Monster Hunter World
          374320, #Dark Souls III
          648800, #Raft
          960090, #Bloon TD 6
          1063730, #New World
          322170, #Geometry Dash
          250900, #The Binding of Isaac: Rebirth
          433850, #Z1 Battle Royale
          1145360, #Hades
          264710, #Subnautica
          1938090, #Call of Duty Modern Warfare II
          275850, #No Mans Sky
          238960, #Path of Exile
          377160, #Fallout 4
          438100, #VRChat
          1599340, #Lost Ark
          289070, #Sid Meier's Civilization VI
          698780, #Doki Doki Literature Club
          976730, #Halo: The Master Chief Collection
          1468810, #Tale of Immortal
          1794680, #Vampire Survivors
          49520, #Borderlands 2
          72850, #The Elder Scrolls V: Skyrim
          251570, #7 Days to Die
          391540, #Undertale
          261550, #Mount & Blade II: Bannerlord
          255710, #Cities: Skylines
          1118200, #People Playground
          814380, #Sekiro: Shadows Die Twice
          1293830, #Forza Horizon 4
          394360, #Hearts of Iron IV
          548430, #Deep Rock Galactic
          319630, #Life is Strange
          700330, #SCP: Secret Laboratory
          1240440, #Halo Infinite
          284160, #BeamNG.drive
          632360, #Risk of Rain 2
          107410, #Arma 3
          782330, #Doom Eternal
          1782210, #Crab Game
          108600, #Project Zomboid
          1625450, #Muck
          227940, #Heroes & Generals
          990080, #Hogwarts Legacy
          22380, #Fallout: New Vegas
          477160, #Human: Fall Flat
          435150, #Divinity: Original Sin 2
          1517290, #Battlefield 2042
          1237970, #Titanfall 2
          203160, #Tomb Raider
          10, #Counter-Strike
          294100, #RimWorld
          1238810, #Battlefield V
          427520, #Factorio
          489830, #The Elder Scross V: Skyrim Special Edition
          1203220, #Naraka: Bladepoint
          444200, #World of Tanks Blitz
          220, #Half-Life 2
          594650, #Hunt: Showdown
          552520, #Far Cry 5
          812140, #Assassins Creed Odyssey
          552990, #World of Warships
          268910, #Cuphead
          301520,#Robocraft
          379720,#Doom
          48700,#Mount & Blade: Warband
          8930,#Side Meier's Civilization V
          1832640, #Mirror 2: Project X
          588650, #Dead Cells
          646570,#Slay the Spire
          1289310,#Helltaker
          526870,#Satisfactory
          1326470,#Sons of the Forest
          306130,#The Elder Scrolls Online
          400,#Portal
          677620,#Splitgate
          518790,#theHunter: Call of the Wild
          386360,#SMITE
          262060,#Darkest Dungeon
          813780,#Age of Empires II: Definitive Edition
          1426210, #It Takes Two
          1172380, #STAR WARS Jedi: Fallen Order
          1551360, #Forza Horizon 5
          12210, #Grand Theft Auto IV: The Complete Edition
          1332010, #Stray
          534380, #Dying Light 2 Stay Human
          281990, #Stellaris
          240, #Counter-Strike: Source
          755790, #Ring of Elysium
          270880, #American Truck Simulator
          1057090, #Ori and the Will of the Wisps
          393380, #Squad
          508440, #Totally Accurate Battle Simulator
          8870, #BioShock Infinite
          1506830, #Fifa 22
          311210, #Call of Duty: Black Ops III
          1238840, #Battlefield 1
          1222670, #The Sims 4
          3590,#Plants vs Zombies GOTY Edition
          391220,#Rise of the Tomb Raider
          433340,#Slime Rancher
          1818750,#MultiVersus
          457140,#Oxygen Not Included
          220200,#Kerbal Space Program
          397540,#Borderlands 3
          361420,#ASTRONEER
          1046930,#Dota Underlords
          304050,#Trove
          225540,#Just Cause 3
          362890,#Black Mesa
          629760,#MORDHAU
          387990,#Scrap Mechanic
          466240,#Deceit
          674940,#Stick Fight: The Game
          714010,#Aim Lab
          222880,#Insurgency
          883710,#Resident Evil 2
          219740,#Don't Starve
          305620,#The Long Dark
          1811260,#EA Sports Fifa 23
          524220,#NieR: Automata
          1144200,#Ready or Not
          581320,#Insurgency: Sandstorm
          211820,#Starbound
          1092790,#Inscryption
          220240,#Far Cry 3
          291480,#Warface
          1190460,#Death Stranding
          594570,#Total War: WARHAMMER II
]

base_url = 'https://store.steampowered.com/appreviews/'
#limiting to english reviews
query_params = {
    'json': 1,
    'num_per_page': 100,
    'language': 'english',
    'filter': 'updated',
}

In [17]:
### Old Approach ### 
#get a list of app ids
def save_game_dlc_ids():
    url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
    data = requests.get(url).json()
    app_ids = [entry['appid'] for entry in data['applist']['apps']]
    print('Total number of app ids: ' + str(len(app_ids)))
    
    #filter out non-game apps
    url = 'https://store.steampowered.com/api/appdetails'
    game_ids = []
    dlc_ids = []
    for app_id in tqdm.tqdm(app_ids):
        params = {
            'appids': app_id['appid'],
            'filters': 'basic'
        }
        data = requests.get(url, params=params).json()
        if (data[str(app_id['appid'])]['success']):
            if (data[str(app_id['appid'])]['data']['type'] == 'game'):
                game_ids.append(app_id['appid'])
            elif (data[str(app_id['appid'])]['data']['type'] == 'dlc'):
                dlc_ids.append(app_id['appid'])
        time.sleep(0.5)
    print('Total number of game app ids: ' + str(len(game_ids)))
    print('Total number of dlc app ids: ' + str(len(dlc_ids)))
    
    #pickle game ids and dlc ids
    with open('game_ids.pickle', 'wb') as f:
        pickle.dump(game_ids, f)
    with open('dlc_ids.pickle', 'wb') as f:
        pickle.dump(dlc_ids, f)

def filter_app_ids_by_review_count(app_ids, target_positive, target_negative):
    #filter out games that don't have enough positive or negative reviews
    for app_id in app_ids:
        data = requests.get(base_url + str(app_id['appid']), params=query_params).json()
        if (data['query_summary']['total_positive'] < target_positive or data['query_summary']['total_negative'] < target_negative):
            app_ids.remove(app_id)
        time.sleep(0.5)    

    print('Total number of game app ids with enough reviews: ' + str(len(app_ids)))
    #save app ids to file
    file_name = 'app_ids_' + str(target_positive) + '_' + str(target_negative) + '.pickle'
    with open(file_name, 'wb') as f:
        pickle.dump(app_ids, f)
    return app_ids

def load_app_ids():
    with open('app_ids.pickle', 'rb') as f:
        return pickle.load(f)

In [31]:
#path of positive files: steam/pos
#path of negative files: steam/neg

#make directories
os.makedirs('steam/pos', exist_ok=True)
os.makedirs('steam/neg', exist_ok=True)

#set target number of reviews per each game
target_positive = 200
target_negative = 200

def filter_by_number_of_reviews(app_ids, target_positive, target_negative):
    valid_app_ids = []
    for app_id in tqdm.tqdm(app_ids):
        response = requests.get(base_url + str(app_id), params=query_params)
        if response.status_code == 200:
            response_json = response.json()
            if response_json['success'] == 1:
                number_of_positive_reviews = response_json['query_summary']['total_positive']
                number_of_negative_reviews = response_json['query_summary']['total_negative']
                if number_of_positive_reviews >= target_positive and number_of_negative_reviews >= target_negative:
                    valid_app_ids.append(app_id)
        time.sleep(0.25)
    return valid_app_ids

#Steam reviews can be in ascii or in unicode, so try to convert if possible        
def convert_to_ascii(review_text):
    try:
        return True, unidecode.unidecode(review_text, errors='strict')
    except(UnidecodeError):
        return False, None

#write reviews to files (return bool for if file was written or not)
def write_file(review_json_data, app_id):
    filename = review_json_data['recommendationid']
    review_text = review_json_data['review']
    recommend = review_json_data['voted_up']
    conversion_success, review_text = convert_to_ascii(review_text)
    if conversion_success:
        try:
            if recommend:
                with open('steam/pos/' + app_id + "/" + filename + ".txt", 'x') as f:
                    f.write(review_text)
            else:
                with open('steam/neg/' + app_id + "/" + filename + ".txt", 'x') as f:
                    f.write(review_text)
        except(FileExistsError):
            #file already exists for some reason
            return False
        return True
    else:
        return False
    
def scrape_review(review_type, app_id, target_number):
    '''
    Scrapes target number of reviews of a specific type from a specific game, saving them to files.

    Args:
        review_type: 'positive' or 'negative'
        app_id: app id of game
        target_number: target number of reviews to scrape

    Returns:
        None
    '''
    query_params['cursor'] = '*'
    query_params['review_type'] = review_type
    current_reviews = 0

    with tqdm.tqdm(total=target_number, leave=False) as progress_bar:
        while current_reviews < target_number:
            data = requests.get(base_url + str(app_id), params=query_params).json()
            
            #error checking
            if (data['success'] != 1):
                print('Error: ' + data['error'])
                break
            
            #check if there are reviews
            if (len(data['reviews']) == 0):
                print('Warning: Out of reviews for app id ' + str(app_id))
                print(data)
                break

            tqdm_count = 0
            for review in data['reviews']:
                if write_file(review, str(app_id)):
                    current_reviews += 1
                    tqdm_count += 1
                if current_reviews >= target_number:
                    break
            #update cursor (need to url encode)
            query_params['cursor'] = data['cursor']
            progress_bar.update(tqdm_count)
            #warn if no reviews were written
            if (tqdm_count == 0):
                print('Warning: Out of reviews for app id ' + str(app_id))
                break
            #sleep to avoid rate limiting
            time.sleep(1)
        #warn if we didn't get enough reviews
        if current_reviews < target_number:
            print('Warning: Only got ' + str(current_reviews) + ' reviews for app id ' + str(app_id))

def main_scraping(appids):
    for app_id in appids:
        print('Scraping reviews for app id: ' + str(app_id))
        os.makedirs('steam/pos/' + str(app_id), exist_ok=True)
        os.makedirs('steam/neg/' + str(app_id), exist_ok=True)
        #get positive reviews
        scrape_review('positive', app_id, target_positive)

        #get negative reviews
        scrape_review('negative', app_id, target_negative)

In [23]:
print(len(appids))
valid_appids = filter_by_number_of_reviews(appids, target_positive, target_negative)
print(len(valid_appids))

167


100%|██████████| 167/167 [01:27<00:00,  1.90it/s]

150


In [32]:
shutil.rmtree('steam')
main_scraping(valid_appids)

Scraping reviews for app id: 730


Scraping reviews for app id: 578080


Scraping reviews for app id: 570


Scraping reviews for app id: 271590


Scraping reviews for app id: 359550


...

Scraping reviews for app id: 1092790


Scraping reviews for app id: 220240


Scraping reviews for app id: 291480


Scraping reviews for app id: 1190460


Scraping reviews for app id: 594570


In [33]:
#shuffle into train, validation, and test sets 
#(80% train, 10% validation, 10% test)
#make directories
import random

shutil.rmtree('steam-ds')

os.makedirs('steam-ds/train/pos', exist_ok=True)
os.makedirs('steam-ds/train/neg', exist_ok=True)
os.makedirs('steam-ds/val/pos', exist_ok=True)
os.makedirs('steam-ds/val/neg', exist_ok=True)
os.makedirs('steam-ds/test/pos', exist_ok=True)
os.makedirs('steam-ds/test/neg', exist_ok=True)

#fix random seed
random.seed(777)

#shuffle and split
for app_id in tqdm.tqdm(valid_appids):
    #get all positive reviews
    pos_reviews = os.listdir('steam/pos/' + str(app_id))
    random.shuffle(pos_reviews)
    #get all negative reviews
    neg_reviews = os.listdir('steam/neg/' + str(app_id))
    random.shuffle(neg_reviews)
    #split into train, validation, and test sets
    pos_train = pos_reviews[:int(len(pos_reviews) * 0.8)]
    pos_val = pos_reviews[int(len(pos_reviews) * 0.8):int(len(pos_reviews) * 0.9)]
    pos_test = pos_reviews[int(len(pos_reviews) * 0.9):]
    neg_train = neg_reviews[:int(len(neg_reviews) * 0.8)]
    neg_val = neg_reviews[int(len(neg_reviews) * 0.8):int(len(neg_reviews) * 0.9)]
    neg_test = neg_reviews[int(len(neg_reviews) * 0.9):]
    #move files
    for filename in pos_train:
        shutil.copy('steam/pos/' + str(app_id) + '/' + filename, 'steam-ds/train/pos/' + filename)
    for filename in pos_val:
        shutil.copy('steam/pos/' + str(app_id) + '/' + filename, 'steam-ds/val/pos/' + filename)
    for filename in pos_test:
        shutil.copy('steam/pos/' + str(app_id) + '/' + filename, 'steam-ds/test/pos/' + filename)
    for filename in neg_train:
        shutil.copy('steam/neg/' + str(app_id) + '/' + filename, 'steam-ds/train/neg/' + filename)
    for filename in neg_val:
        shutil.copy('steam/neg/' + str(app_id) + '/' + filename, 'steam-ds/val/neg/' + filename)
    for filename in neg_test:
        shutil.copy('steam/neg/' + str(app_id) + '/' + filename, 'steam-ds/test/neg/' + filename)

#zip files
shutil.make_archive('steam-ds', 'zip', 'steam-ds')

100%|██████████| 150/150 [28:46<00:00, 11.51s/it]


'/mnt/c/Users/Sandaru/OneDrive/Sandaru/School/college/Senior S2/Natural Language Processing CS 7650/Group Project/steam-ds.zip'